In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [2]:
# getting the class from the Tree file
%run Tree.ipynb

In [3]:
raw_data = pd.read_csv("../IRIS.csv")
#extracting unique species
types = pd.unique(raw_data["species"])

idx = [i for i in range(len(types))]
data = raw_data.copy()
data["species"].replace(types,idx,inplace=True)

In [4]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=10, random_state=31)

## The Random Forest Class

In [13]:
class RandomForest():
    def __init__(self, n_features=None, n_trees=10, max_depth=3, min_split_sample=2, gain_fn="gini"):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_split_sample = min_split_sample
        self.n_features = n_features
        self.gain_fn = gain_fn
        self.forest=[]
        
    def _create_forest(self,X,Y):
        
        for _ in range(self.n_trees):
            print(_)
            Classifier = Decision_tree(max_depth=self.max_depth, min_sample_split=self.min_split_sample, 
                                       gain_fn=self.gain_fn,n_features=self.n_features)
            sampled_X, sampled_Y = self.bootstrap(X,Y)
            print(sampled_X.shape)
            Classifier.fit(sampled_X, sampled_Y)
            self.forest.append(Classifier)
            
    
    def bootstrap(self, X, Y):
        
        n_samples = X.shape[0]
        choosen_idx = np.random.choice(n_samples, n_samples, replace=True)
        return X[choosen_idx], Y[choosen_idx]
        
    def fit(self, X, Y):
        self._create_forest(X,Y)
        
    def most_common_label(self, pred):
        counter = Counter(pred)
        return counter.most_common(1)[0][0]
    
    def predict(self, X, tree_pred=False):
        
        pred = np.array([tree.predict(X) for tree in self.forest])
        tree_preds = np.swapaxes(pred, 0, 1)
        pred = np.array([self.most_common_label(pred) for pred in tree_preds])
        
        return pred
    
        
        

In [14]:
forest = RandomForest(n_trees=3)
forest.fit(X_train,Y_train)

0
(140, 4)
1
(140, 4)
2
(140, 4)


In [15]:
lables_test = forest.predict(X_test)
lables_train = forest.predict(X_train)

In [17]:
test_acc = accuracy_score(lables_test, Y_test)
train_acc = accuracy_score(lables_train, Y_train)

print(f"The Training accuracy is {train_acc*100:0.2f}")
print(f"The Testing accuracy is {test_acc*100:0.2f}")

The Training accuracy is 88.57
The Testing accuracy is 80.00
